# PF Inventory Projections: Q3-Q4 '23
#### Adrafül Labs

**Notes:**

Inventory will be projected using a combination of:
- increased by imported inventory addition data
- decreased by projected sales data that can be found in sales.ipynb

Charts will be produced using the plotly charting library.

All future projections are estimates.

_____________

**Starting Data**

We first read in all external data that will be used for projections.

In [10]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [11]:
# import sales data
sales_df = pd.read_excel('../../data/outputs/projected_daily_sales.xlsx')
sales_df.drop('Unnamed: 0', axis=1, inplace=True)
# import inventory data
inventory_df = pd.read_excel('../../data/imports/excel/inventory_projected_purchases.xlsx')
projected_inventory_df = pd.DataFrame(columns=['date', 'inventory'])
projected_inventory_df['date'] = pd.date_range(start='2023-08-01', end='2023-12-31')
projected_inventory_df['inventory'] = 0
merged_df = projected_inventory_df.merge(inventory_df, on='date', how='left')
merged_df['inventory'] = merged_df['purchase_quantity'].cumsum()
merged_df['inventory'].fillna(method='ffill', inplace=True)
merged_df.drop(columns='purchase_quantity', inplace=True)
projected_inventory_df['inventory'] = merged_df['inventory']

We projected the inventory purchases based on a target inventory buffer and our projected sales. In this case, we will be forced to purchase a larger quantity of inventory up front to permit our relatively bullish sales projections. Below are the projected inventory levels after purchases and before application of sales.

In [12]:
inventory_fig = go.Figure(data=go.Bar(x=projected_inventory_df['date'], y=projected_inventory_df['inventory']))
inventory_fig.update_layout(plot_bgcolor='#ece9f1')
inventory_fig.show()

See the purchase schedule below:

In [13]:
inventory_df.head(10)

,date,purchase_quantity,purchase_amount
0,2023-08-01,10000,20400
1,2023-09-01,25000,51000
2,2023-10-01,37500,71250
3,2023-11-01,50000,92500
4,2023-12-01,50000,90000


**Inventory Costing**

We can cost the inventory based on the projected CPU for this case.

In [14]:
# CPU taken from local data (calculated with production calculator)
cpu_df = pd.read_excel('../../data/imports/excel/projected_cpu.xlsx')
projected_inventory_df['value'] = projected_inventory_df['inventory'] * cpu_df['projected_cpu']
inventory_fig = go.Figure(data=go.Bar(x=projected_inventory_df['date'], y=projected_inventory_df['inventory'], name='units'))
inventory_fig.add_trace(go.Scatter(x=projected_inventory_df['date'], y=projected_inventory_df['value'], mode='lines', name='value'))
inventory_fig.update_layout(plot_bgcolor='#ece9f1')
inventory_fig.update_layout(title='Projected Inventory & Value', xaxis_title='Date', yaxis_title='Units')
inventory_fig.show()

**Application of Sales**

The daily change in inventory will be equivalent to daily sales / average selling PPU. We will now apply the daily sales to inventory to project stock levels over time.

$daily\_inventory\_change_t$ = -($total\_gross\_unit\_sales_t$ $\div$ $PPU$)

In [15]:
PPU = (3.915 * .4) + (3.37 * .6)
print(f"Average PPU: {PPU:.2f}")

Average PPU: 3.59


In [16]:
sales_df['units_sold'] = sales_df['total_gross_sales'] / PPU
sales_df['units_sold'] = sales_df['units_sold'].astype(int)
sales_cumsum = np.cumsum(sales_df['units_sold'])
projected_inventory_df['inventory'] = projected_inventory_df['inventory'] - sales_cumsum
projected_inventory_df['value'] = projected_inventory_df['inventory'] * cpu_df['projected_cpu']

In [17]:
updated_inventory_fig = go.Figure(data=go.Bar(x=projected_inventory_df['date'], y=projected_inventory_df['inventory'], name='units'))
updated_inventory_fig.add_trace(go.Scatter(x=projected_inventory_df['date'], y=projected_inventory_df['value'], mode='lines', name='value'))
inventory_fig.update_layout(plot_bgcolor='#ece9f1')
updated_inventory_fig.update_layout(title='Projected Inventory', xaxis_title='Date', yaxis_title='Units')
updated_inventory_fig.show()

_______

In [9]:
# Exporting Data

projected_inventory_df.to_excel('../../data/outputs/projected_daily_inventory.xlsx')